In [2]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import time
from shapely.geometry import box

# --- 1. Baixar o polígono do arthur alvim ---
bairro = ox.geocode_to_gdf("SAPOPEMBA, São Paulo")

# --- 2. Criar a grade (grid) sobre o bairro ---
def criar_grid(bairro, tamanho=0.005):  # Aproximadamente 200m
    minx, miny, maxx, maxy = bairro.total_bounds
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            grid.append(box(x, y, x + tamanho, y + tamanho))
            y += tamanho
        x += tamanho
    grid_gdf = gpd.GeoDataFrame(geometry=grid, crs=bairro.crs)
    grid_gdf = gpd.overlay(grid_gdf, bairro, how='intersection')
    return grid_gdf

grid = criar_grid(bairro)


# --- 3. Corrigir projeção para metros, calcular centroide ---
grid = grid.to_crs(epsg=3857)  # Web Mercator
grid["centroide"] = grid.geometry.centroid

# --- 4. Criar colunas separadas de latitude/longitude
centroides = grid["centroide"].to_crs(epsg=4326)  # voltar para lat/lon
grid["centroide_lon"] = centroides.geometry.x
grid["centroide_lat"] = centroides.geometry.y

# --- 6. Definir hospital e API ---
API_KEY = "5b3ce3597851110001cf624824d680a2a89c4dc48933794aaca7b81d"  # Troque aqui pela sua chave
hospital_coords = [-46.5145716, -23.5875541]



def calcular_distancia_km(origem, destino, api_key):
    url = "https://api.openrouteservice.org/v2/directions/driving-car/json"
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    body = {
        "coordinates": [origem, destino]
    }
    try:
        response = requests.post(url, json=body, headers=headers)
        if response.status_code == 200:
            dados = response.json()
            distancia_metros = dados['routes'][0]['summary']['distance']
            distancia_km = distancia_metros / 1000
            return round(distancia_km, 2)
        else:
            print("Erro na API:", response.text)
            return None
    except Exception as e:
        print(f"Erro: {e}")
        return None

# --- 7. Calcular a distância para cada grid ---
distancias_km = []
for i, row in grid.iterrows():
    origem = [row["centroide_lon"], row["centroide_lat"]]
    distancia = calcular_distancia_km(origem, hospital_coords, API_KEY)
    distancias_km.append(distancia)
    time.sleep(1)  # evita limite da API gratuita

grid["distancia_km"] = distancias_km
grid.to_csv(f"estimativa_sapopemba.csv", index=False)

In [3]:
grid

,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,...,place_rank,importance,addresstype,name,display_name,geometry,centroide,centroide_lon,centroide_lat,distancia_km
0,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","POLYGON ((-5179843.739 -2704229.455, -5179843....",POINT (-5180076.765 -2704379.196),-46.533421,-23.597017,3.45
1,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","POLYGON ((-5179843.739 -2703622.087, -5179843....",POINT (-5180040.764 -2703959.827),-46.533098,-23.593565,2.75
2,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","POLYGON ((-5179843.739 -2703014.743, -5179843....",POINT (-5179934.044 -2703334.439),-46.532139,-23.588416,2.33
3,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","POLYGON ((-5179843.739 -2703014.743, -5180069....",POINT (-5179973.343 -2702876.773),-46.532492,-23.584648,2.19
4,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","MULTIPOLYGON (((-5179287.141 -2705143.837, -51...",POINT (-5179314.351 -2704952.439),-46.526572,-23.601736,3.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","POLYGON ((-5174834.362 -2706051.696, -5175390....",POINT (-5175098.502 -2705832.634),-46.488701,-23.608981,5.60
64,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","POLYGON ((-5174834.362 -2707266.64, -5174504.0...",POINT (-5174667.185 -2707416.25),-46.484826,-23.622016,8.10
65,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","POLYGON ((-5174834.362 -2707266.64, -5174834.3...",POINT (-5174685.546 -2706977.123),-46.484991,-23.618401,7.97
66,-46.536328,-23.625784,-46.483282,-23.583295,8103136,relation,3049450,-23.604326,-46.509885,boundary,...,18,0.38624,suburb,Sapopemba,"Sapopemba, São Paulo, Região Imediata de São P...","POLYGON ((-5174834.362 -2706659.156, -5174834....",POINT (-5174764.821 -2706417.247),-46.485703,-23.613793,6.92


In [4]:
%pip install folium
import folium
from shapely.geometry import mapping

# Converter o bairro para EPSG:4326 (lat/lon) para exibição no mapa
bairro_wgs84 = bairro.to_crs(epsg=4326)
grid_wgs84 = grid.to_crs(epsg=4326)

# Criar mapa centralizado no bairro
centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
m = folium.Map(location=[centroide_bairro.y, centroide_bairro.x], zoom_start=14)

# Adicionar polígono do bairro
folium.GeoJson(bairro_wgs84.geometry.__geo_interface__,
               name="Bairro - Tatuape",
               style_function=lambda x: {"color": "black", "fillOpacity": 0.1}
).add_to(m)

# Adicionar os grids
for _, row in grid_wgs84.iterrows():
    folium.GeoJson(mapping(row.geometry),
                   style_function=lambda x: {"color": "blue", "weight": 1, "fillOpacity": 0}
    ).add_to(m)

# Adicionar ponto do hospital
, 
folium.Marker(
    location=[-23.5875541, -46.5145716] ,
    popup="Hospital Sapopemba",
    icon=folium.Icon(color='red', icon='plus-sign')
).add_to(m)

# Mostrar mapa
m

# [-46.5315852, -23.5605378] 


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: folium in c:\users\joth1\appdata\local\programs\python\python312\lib\site-packages (0.19.5)



C:\Users\joth1\AppData\Local\Temp\ipykernel_28756\606786754.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
